In [2]:
import pandas as pd
from numpy import isnan
import vlc
import json
import urllib.request
import pickle

In [39]:
catalanBirds=pd.read_csv("Data/table.csv", sep=",")
scientificToCatalan={}
for row in catalanBirds.iterrows():
    scientificToCatalan[row[1]["Genus"].lower()+" "+row[1]["Species (Scientific)"].lower()]=row[1]["Nom català"]

In [43]:
scientificToCatalan

{'tetrastes bonasia': 'Grevol',
 'tetrao urogallus': 'Gall fer',
 'lagopus muta': 'Perdiu blanca',
 'alectoris rufa': 'Perdiu roja',
 'perdix perdix': 'Perdiu xerra',
 'coturnix coturnix': 'Guatlla',
 'phasianus colchicus': 'Faisà',
 'branta bernicla': 'Oca de collar',
 'branta canadensis': 'Oca del Canadà',
 'branta leucopsis': 'Oca de galta blanca',
 'anser indicus': 'Oca índia',
 'anser anser': 'Oca vulgar',
 'anser fabalis': 'Oca pradenca',
 'anser brachyrhynchus': 'Oca de bec curt',
 'anser albifrons': 'Oca riallera grossa',
 'cygnus atratus': 'Cigne negre',
 'cygnus olor': 'Cigne mut',
 'cygnus columbianus': 'Cigne petit',
 'cygnus cygnus': 'Cigne cantaire',
 'alopochen aegyptiaca': 'Oca egípcia',
 'tadorna tadorna': 'Ànec blanc',
 'tadorna ferruginea': 'Ànec canyella',
 'aix galericulata': 'Ànec mandarí',
 'spatula querquedula': 'Xarrasclet',
 'spatula discors': 'Xarxet alablau',
 'spatula clypeata': 'Ànec cullerot',
 'mareca strepera': 'Ànec griset',
 'mareca falcata': 'Xarxet 

In [48]:
allbirds=pd.read_csv("Data/AvesEspaña.csv", header=None, sep=";")

# Get column names
column_names=["sp_latin", "ssp", "sp_ingles", "sp_espanol", "PB", "NA", "CA", "rareza"]
allbirds.columns=column_names

# Take family name and put it as new column
family=[]
lastfamily=None
for row in allbirds.iterrows():
    if not isinstance(row[1][3], str):
        lastfamily=row[1][0].lower()
    else:
        family.append(lastfamily)
        
# Remove family rows and set them as new column
allbirds=allbirds[allbirds["sp_ingles"].apply(lambda x : isinstance(x, str))]
allbirds["family"]=family

# lowercase all scientific names
allbirds["sp_latin"]=allbirds["sp_latin"].apply(lambda x : x.lower())

def toCatalan(x):
    if x not in scientificToCatalan.keys():
        return "noName"
    else:
        return scientificToCatalan[x]
    
# Set catalan name as new column
allbirds["sp_cat"]=allbirds["sp_latin"].apply(toCatalan)

# Filter all birds for which we don't know the catalan name
catBirds=allbirds[allbirds["sp_cat"]!="noName"]
catBirds.head()

572


,sp_latin,ssp,sp_ingles,sp_espanol,PB,NA,CA,rareza,family,sp_cat
1,branta bernicla,"bernicla, hrota",Brant Goose,Barnacla carinegra,A,NaN,A,R2 (solo ssp. hrota).,anatidae,Oca de collar
3,branta leucopsis,NaN,Barnacle Goose,Barnacla cariblanca,A (i),NaN,D,NaN,anatidae,Oca de galta blanca
5,anser anser,anser,Greylag Goose,Ánsar común,A (i),A,A(i),NaN,anatidae,Oca vulgar
6,anser fabalis,"fabalis, rossicus",Bean Goose,Ánsar campestre,A,NaN,NaN,"R1 (ssp. fabalis), R2 (ssp. rossicus y sin asi...",anatidae,Oca pradenca
7,anser brachyrhynchus,NaN,Pink-footed Goose,Ánsar piquicorto,A,NaN,A,NaN,anatidae,Oca de bec curt


In [49]:
def make_url(bird_sp):
    '''Given the scientific name of a bird species this function
       returns the url to retrieve the xeno-canto recordings '''
    
    query="https://www.xeno-canto.org/api/2/recordings?query="
    genus, name = bird_sp.split(" ")
    completeQuery=query+genus+"+"+name+"+q:A"
    return completeQuery

def get_recordings(url):
    '''Makes a request to retrieve a list of all the recordings 
       for the parameters specified in url'''
    
    with urllib.request.urlopen(url) as response:
        data = json.loads(response.read().decode())
        return data["recordings"]
    
bird="branta bernicla"
url=make_url(bird)
get_recordings(url)

[{'id': '631977',
  'gen': 'Branta',
  'sp': 'bernicla',
  'ssp': 'nigricans',
  'en': 'Brant Goose',
  'rec': 'Steve Hampton',
  'cnt': 'United States',
  'loc': 'Pt Hudson, Port Townsend, Jefferson County, Washington',
  'lat': '48.1176',
  'lng': '-122.7494',
  'alt': '0',
  'type': 'call',
  'url': '//www.xeno-canto.org/631977',
  'file': '//www.xeno-canto.org/631977/download',
  'file-name': 'XC631977-Brant Mar 25b.mp3',
  'sono': {'small': '//www.xeno-canto.org/sounds/uploaded/BUGKVZBWKF/ffts/XC631977-small.png',
   'med': '//www.xeno-canto.org/sounds/uploaded/BUGKVZBWKF/ffts/XC631977-med.png',
   'large': '//www.xeno-canto.org/sounds/uploaded/BUGKVZBWKF/ffts/XC631977-large.png',
   'full': '//www.xeno-canto.org/sounds/uploaded/BUGKVZBWKF/ffts/XC631977-full.png'},
  'lic': '//creativecommons.org/licenses/by-nc-sa/4.0/',
  'q': 'A',
  'length': '0:23',
  'time': '19:00',
  'date': '2021-03-25',
  'uploaded': '2021-03-26',
  'also': ['Larus glaucescens'],
  'rmk': 'group of 225 bir

In [54]:
#birdRecBySp={}
# for i, bird in enumerate(allbirds["sp_latin"]):
#     print(100*(i/len(allbirds)))
#     if len(bird.split(" "))==2:
#         birdRecBySp[bird]=get_recordings(make_url(bird))

# birdRecBySpCat={}
# for i, bird in enumerate(catBirds["sp_latin"]):
#     print(100*(i/len(catBirds)))
#     if len(bird.split(" "))==2:
#         birdRecBySpCat[bird]=get_recordings(make_url(bird))

# with open('Data/birdRecBySpCatDict.pickle', 'wb') as handle:
#     pickle.dump(birdRecBySpCat, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('Data/birdRecBySpCatDict.pickle', 'rb') as handle:
   birdRecBySpCat = pickle.load(handle)

In [55]:
url=birdRecBySpCat["troglodytes troglodytes"][8]["file"]
url

'//www.xeno-canto.org/643490/download'

In [56]:
includedBirds=list(birdRecBySpCat.keys())
print(includedBirds)

['branta bernicla', 'branta leucopsis', 'anser anser', 'anser fabalis', 'anser brachyrhynchus', 'anser albifrons', 'cygnus olor', 'cygnus columbianus', 'cygnus cygnus', 'tadorna tadorna', 'tadorna ferruginea', 'spatula querquedula', 'spatula discors', 'spatula clypeata', 'mareca strepera', 'mareca penelope', 'anas platyrhynchos', 'anas rubripes', 'anas acuta', 'anas crecca', 'anas carolinensis', 'marmaronetta angustirostris', 'netta rufina', 'aythya ferina', 'aythya nyroca', 'aythya collaris', 'aythya fuligula', 'aythya marila', 'aythya affinis', 'somateria spectabilis', 'somateria mollissima', 'melanitta fusca', 'melanitta nigra', 'clangula hyemalis', 'bucephala clangula', 'mergellus albellus', 'mergus merganser', 'mergus serrator', 'oxyura leucocephala', 'tetrastes bonasia', 'tetrao urogallus', 'lagopus muta', 'alectoris rufa', 'perdix perdix', 'coturnix coturnix', 'gavia stellata', 'gavia arctica', 'gavia immer', 'hydrobates pelagicus', 'oceanodroma leucorhoa', 'calonectris diomedea

In [116]:
import random as rand
scientificToCatalan[rand.choice(includedBirds)]

'Fumarell negre'

In [118]:
def test(numQuestions):
    for i in range(numQuestions):
        spLatin=rand.choice(includedBirds)
        url="https:"+rand.choice(birdRecBySpCat[spLatin])["file"]
        question(url, spLatin)

def question(url, bird=None, possible=None):
    if possible is not None:
        print("Les teves opcions són:")
        print(possible)
    p = vlc.MediaPlayer(url)
    p.play()
    print("Enter bird name:")
    res=bird
    ans=input()
    if ans == res:
        print("Bieeeen")
    else:
        print("maaaaal")
        print("Resposta correcta:", bird)
    p.set_pause(1)
#url="https://www.xeno-canto.org/sounds/uploaded/ZWAQHOJFLZ/XC477551-190503-Troglodyte%20mignon%40Sacharewo.mp3"
#question("https:"+url, "Cargolet")
test(5)

Enter bird name:
dendrocopus major
maaaaal
Resposta correcta: dryobates minor
Enter bird name:
uq
maaaaal
Resposta correcta: falco eleonorae
Enter bird name:
wrwer
maaaaal
Resposta correcta: galerida theklae
Enter bird name:
anser anser
maaaaal
Resposta correcta: anser fabalis
Enter bird name:
qwert
maaaaal
Resposta correcta: calidris pusilla
